In [ ]:
%config InlineBackend.figure_formats = ['png']
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import glob
from scipy.optimize import curve_fit
import colorcet as cc
import xarray as xr
import pandas as pd
import os
from scipy.stats import linregress
from numba import njit

plt.style.use(["default"])
plt.rcParams["axes.grid"] = False
plt.rcParams["pdf.fonttype"] = 42  # fontes incorporées en pdf
plt.rcParams["svg.fonttype"] = (
    "none"  # texte en fontes et pas en chemin (comportement par défaut de matplotlib) : il faut installer les polices latex pour pouvoir afficher le fichier correctement dans inkscape
)
plt.rcParams["figure.constrained_layout.use"] = True
plt.rcParams["figure.figsize"] = [86 / 25.4, 86 / 25.4]  # 86 mm : largeur de colonne typique (+/- 6 mm)
plt.rcParams["figure.dpi"] = 160
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Arial"
plt.rcParams["font.size"] = 8
plt.rcParams["mathtext.default"] = "it"
plt.rcParams["axes.linewidth"] = 0.5
plt.rcParams["xtick.minor.width"] = 0.5
plt.rcParams["xtick.major.width"] = 0.5
plt.rcParams["ytick.minor.width"] = 0.5
plt.rcParams["ytick.major.width"] = 0.5
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["lines.linewidth"] = 0.5
plt.rcParams["axes.labelpad"] = 1
plt.rcParams["xtick.minor.pad"] = 1.4
plt.rcParams["xtick.major.pad"] = 1.5
plt.rcParams["ytick.minor.pad"] = 1.4
plt.rcParams["ytick.major.pad"] = 1.5
plt.rcParams["legend.frameon"] = False
plt.rcParams["legend.fancybox"] = False
plt.rcParams["savefig.bbox"] = "tight"
plt.rcParams["savefig.pad_inches"] = 0
plt.rcParams["savefig.transparent"] = False  # True for figure production
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=cc.cm.glasbey_bw_minc_20_maxl_70.colors
)  # couleurs de colorcet et pas de matplotlib

# CLEO data

In [ ]:
d_ef_cleo = np.loadtxt("evaporation_fraction.txt", delimiter=",")

In [ ]:
vav = xr.open_dataset("veritcal_average_cleo_output_evaporation_only_setup.nc")
vav

In [ ]:
vav.relative_humidity.plot()
vav.relative_humidity.mean()

In [ ]:
vav.air_temperature.plot()
vav.air_temperature.mean()

In [ ]:
se = xr.open_dataset("cleo_output_evaporation_only_setup.nc")
se

In [ ]:
se.relative_humidity.mean(dim="cloud_id").plot()

In [ ]:
se.gridbox_coord3.mean(axis=0).plot()

In [ ]:
plt.plot(se.gridbox_coord3.mean(axis=0), se.relative_humidity.mean(dim="cloud_id"))
plt.axvline(x=640)

In [ ]:
plt.plot(
    se.gridbox_coord3.sel(gridbox=slice(0, 30)).mean(axis=0),
    se.relative_humidity.sel(gridbox=slice(0, 30)).mean(dim="cloud_id"),
)
plt.axvline(x=640)

In [ ]:
print(se.relative_humidity.sel(gridbox=slice(0, 1)).mean())
print(se.relative_humidity.sel(gridbox=slice(0, 1)).min())
print(se.relative_humidity.sel(gridbox=slice(0, 1)).max())

print(se.relative_humidity.sel(gridbox=slice(0, 1)).quantile(q=(0.1, 0.9)))

<xarray.DataArray 'relative_humidity' ()> Size: 8B
array(76.41008964)
Coordinates:
    microphysics  <U12 48B ...
<xarray.DataArray 'relative_humidity' ()> Size: 8B
array(64.94911424)
Coordinates:
    microphysics  <U12 48B ...
<xarray.DataArray 'relative_humidity' ()> Size: 8B
array(82.32487039)
Coordinates:
    microphysics  <U12 48B ...
<xarray.DataArray 'relative_humidity' (quantile: 2)> Size: 16B
array([73.87724555, 79.58236078])
Coordinates:
  * quantile  (quantile) float64 16B 0.1 0.9


In [ ]:
se.relative_humidity.plot()

# Physical constants

In [ ]:
rhow = 0.998e3
rhoa = 1.2
eta = 1.85e-5
g = 9.81
nu = eta / rhoa
T = 294.41807507
p = 1e5
Dv0 = 0.211 * (T / 273.15) ** (1.94) * (1013.25e2 / p) * 1e-4  # PK97 (13-3)
Sc = 0.71  # nu/Dv0
gamma = 73e-3
Coo = 0.26
Cgamma = 18.4
lgamma = np.sqrt(gamma / (rhow * g))
kb = 1.380649e-23
Rconst = 8.314
Rv = 461.5
lv = 2.5e6
ka = 26.19e-3


def psat_water(T):
    theta = T - 273.15
    psat = 6.1121e2 * np.exp((18.678 - theta / 234.5) * (theta / (257.14 + theta)))
    return psat


def rhosat_water(T):
    rho = psat_water(T) * 18.01528e-3 / (Rconst * T)
    return rho


Dv = Dv0 / (1 + lv * Dv0 * rhosat_water(T) / (ka * T) * (lv / (Rv * T) - 1))
print(Dv0 / Dv)

3.5957635456780634


# Working code

In [ ]:
class solution:
    """Solution data structure"""

    def __init__(self, t, y, x, evaporated, touched_ground):
        self.t = t
        self.y = y
        self.x = x
        self.evaporated = evaporated
        self.touched_ground = touched_ground


def event_evap(t, y):
    """Function that checks if the mass is below 5% of the initial mass.
    If it is the case, the integration stops and the drop is assumed to be evaporated"""
    if y[0] <= 0.05:
        return True


@njit
def solve_ode(f, time_tuple, y0, dt, max_x, rhosat, rhov):
    """RK4 solver for right hand side f (must return a numpy array). The integration stops either when x reaches the value specified
    by max_x or test_func returns True (evaporation happens)"""
    tmin, tmax = time_tuple[0], time_tuple[1]
    assert tmax > tmin
    assert dt > 0
    N = int((tmax - tmin) / dt)
    y = np.zeros((N, len(y0)))
    t = np.zeros(N)
    x = np.zeros(N)
    t[0] = tmin
    y[0, :] = y0[:]
    for i in range(N - 1):
        # if test_func is not None:
        # if test_func(t[i], y[i, :]):
        if y[i, 0] / y0[0] < 1e-2:
            return (t[:i], y[:i, :], x[:i], True, False)
        if x[i] >= max_x:
            return (t[:i], y[:i, :], x[:i], False, True)
        k1 = f(t[i], y[i, :], rhosat, rhov)
        k2 = f(t[i] + dt / 2, y[i, :] + dt * k1 / 2, rhosat, rhov)
        k3 = f(t[i] + dt / 2, y[i, :] + dt * k2 / 2, rhosat, rhov)
        k4 = f(t[i] + dt, y[i, :] + dt * k3, rhosat, rhov)
        y[i + 1, :] = y[i, :] + dt * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        t[i + 1] = t[i] + dt
        x[i + 1] = dt * y[i, 1] + x[i]
        dt = 1e-8 + np.minimum(
            1e-3 * np.abs(y[i, 0] / (-k1[0])), 1e-3 * np.abs(y[i, 1] / (1e-2 + k1[1]))
        )
    return (t, y, x, False, False)


@njit
def vt(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2) * (1 + Cgamma * (a / lgamma) ** 3) ** (1 / 6)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


@njit
def vtlim(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


@njit
def Cd(a, v):
    """Arguments are the velocity in m/s and the radius in m"""
    return 0.5  # (Coo**(1/2)*(1+Cgamma*(a/lgamma)**3)**(1/6)+(12*nu/(a*np.abs(v)))**(1/2))**2


@njit
def Cd_fullreal(a, v):
    """Arguments are the velocity in m/s and the radius in m"""
    return (
        Coo ** (1 / 2) * (1 + Cgamma * (a / lgamma) ** 3) ** (1 / 6)
        + (12 * nu / (a * np.abs(v))) ** (1 / 2)
    ) ** 2


@njit
def fv(a, v):
    """Arguments are mass and velocity"""
    Re = 2 * a * np.abs(v) / nu
    x = Sc ** (1 / 3) * Re ** (1 / 2)
    if a < 60e-6:
        return 1 + 0.108 * x**2
    else:
        return 0.78 + 0.308 * x


@njit
def func_rhs(t, y, rhosat, rhov):
    """Right hand side of the dynamical system. y[0] radius ; y[1] : v"""
    m = 4 / 3 * np.pi * y[0] ** 3 * rhow
    dr = -Dv * fv(y[0], y[1]) * (rhosat - rhov) / rhow / y[0]
    dv = +g - (3 / 8) * Cd(y[0], y[1]) * (rhoa / rhow) * y[1] ** 2 / y[0]
    return np.array([dr, dv])


@njit
def func_rhs_ideal(t, y, rhosat, rhov):
    """Right hand side of the dynamical system. y[0] radius ; y[1] : v"""
    m = 4 / 3 * np.pi * y[0] ** 3 * rhow
    dr = -Dv * (rhosat - rhov) / rhow / y[0]
    dv = +g - (3 / 8) * 0.5 * (rhoa / rhow) * y[1] ** 2 / y[0]
    return np.array([dr, dv])


@njit
def func_rhsrealreal(t, y, rhosat, rhov):
    """Right hand side of the dynamical system. y[0] radius ; y[1] : v"""
    m = 4 / 3 * np.pi * y[0] ** 3 * rhow
    dr = -Dv * (rhosat - rhov) / rhow / y[0]
    dv = +g - (3 / 8) * Cd_fullreal(y[0], y[1]) * (rhoa / rhow) * y[1] ** 2 / y[0]
    return np.array([dr, dv])


class Simu:
    """Simulation run class. All values that are not dimensionless are in SI units."""

    def __init__(self, a0, rhosat, rhov):
        # conditions initiales
        self.a0 = a0
        self.m0 = 4 * np.pi * rhow * a0**3 / 3
        self.vt0 = vt(a0)
        self.Cdvt = Cd(self.vt0, self.a0)
        self.rhosat = rhosat
        self.rhov = rhov

    def solve(self, H, time, dt):
        """Solves until x = H (m)"""
        self.solution = solution(
            *solve_ode(func_rhs, (0, time), np.array([self.a0, self.vt0]), dt, H, self.rhosat, self.rhov)
        )
        return self.solution

    def plot(self):
        plt.plot(self.solution.t, self.solution.y[0], label="mass")
        plt.plot(self.solution.t, self.solution.y[1], label="vitesse")
        plt.legend(loc="best")


class Simu_ideal:
    """Simulation run class. All values that are not dimensionless are in SI units."""

    def __init__(self, a0, rhosat, rhov):
        # conditions initiales
        self.a0 = a0
        self.m0 = 4 * np.pi * rhow * a0**3 / 3
        self.vt0 = vt(a0)
        self.Cdvt = 0.5
        self.rhosat = rhosat
        self.rhov = rhov

    def solve(self, H, time, dt):
        """Solves until x = H (m)"""
        self.solution = solution(
            *solve_ode(
                func_rhs_ideal, (0, time), np.array([self.a0, self.vt0]), dt, H, self.rhosat, self.rhov
            )
        )
        return self.solution

    def plot(self):
        plt.plot(self.solution.t, self.solution.y[0], label="mass")
        plt.plot(self.solution.t, self.solution.y[1], label="vitesse")
        plt.legend(loc="best")


class Simurealreal:
    """Simulation run class. All values that are not dimensionless are in SI units."""

    def __init__(self, a0, rhosat, rhov):
        # conditions initiales
        self.a0 = a0
        self.m0 = 4 * np.pi * rhow * a0**3 / 3
        self.vt0 = vt(a0)
        self.Cdvt = Cd(self.vt0, self.a0)
        self.rhosat = rhosat
        self.rhov = rhov

    def solve(self, H, time, dt):
        """Solves until x = H (m)"""
        self.solution = solution(
            *solve_ode(
                func_rhsrealreal, (0, time), np.array([self.a0, self.vt0]), dt, H, self.rhosat, self.rhov
            )
        )
        return self.solution

    def plot(self):
        plt.plot(self.solution.t, self.solution.y[0], label="mass")
        plt.plot(self.solution.t, self.solution.y[1], label="vitesse")
        plt.legend(loc="best")

# Size dependance

In [ ]:
<xarray.DataArray 'relative_humidity' ()> Size: 8B
array(76.41008964)
Coordinates:
    microphysics  <U12 48B ...
<xarray.DataArray 'relative_humidity' ()> Size: 8B
array(64.94911424)

max 82.32487039
Coordinates:
    microphysics  <U12 48B ...
<xarray.DataArray 'relative_humidity' (quantile: 2)> Size: 16B
array([73.87724555, 79.58236078])
Coordinates:
  * quantile  (quantile) float64 16B 0.1 0.9

In [ ]:
H = 700  # 1200 #m
RH = 76.41008964 / 100  # 85.61890708/100
RHmin = 64.94911424 / 100
RHmax = 82.32487039 / 100

In [ ]:
r0s = np.logspace(-4, -2.5, 500)
rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu(r0, rhosat_water(T), RH * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef = (r0s**3 - rfs**3) / r0s**3

In [ ]:
for k, r0 in enumerate(r0s):
    s = list_simus[k]
    plt.plot(s.solution.y[:, 0] * 1e6, H - s.solution.x)
plt.semilogx()

[]

In [ ]:
rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu_ideal(r0, rhosat_water(T), RH * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_ideal = (r0s**3 - rfs**3) / r0s**3


plt.plot(1 - rfs / r0s, ef)
xplot = np.linspace(0, 1, 100)
plt.plot(xplot, 0.59 * xplot + 0.18)

In [ ]:
rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu_ideal(r0, rhosat_water(T), RH * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_ideal = (r0s**3 - rfs**3) / r0s**3

rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu_ideal(r0, rhosat_water(T), RHmin * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_rhmin = (r0s**3 - rfs**3) / r0s**3

rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu_ideal(r0, rhosat_water(T), RHmax * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_rhmax = (r0s**3 - rfs**3) / r0s**3

rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simurealreal(r0, rhosat_water(T), RH * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_realreal = (r0s**3 - rfs**3) / r0s**3

In [ ]:
bU = np.sqrt(8 / 3 * rhow / rhoa * g / 0.5)
dr52 = 5 / 2 * Dv * H / bU * (1 - RH) * rhosat_water(T) / rhow
efftheo = 1 - (1 - dr52 / r0s ** (5 / 2)) ** (6 / 5)
fvs = np.array([fv(r0, vtlim(r0)) for r0 in r0s])

plt.plot(d_ef_cleo[:, 0], d_ef_cleo[:, 1], linestyle="", marker="+", markersize=3)
plt.loglog(
    r0s * 1e6,
    100 * efftheo,
    label=r"analytic theory $f_v = 1$",
    linestyle="dashed",
    linewidth=1.5,
    color="green",
)
plt.loglog(r0s * 1e6, 100 * ef_ideal, label=r"$f_v = 1$", color="k")

plt.loglog(r0s * 1e6, 100 * ef_rhmin, label=r"$f_v = 1$ , min. RH", color="blue")
plt.loglog(r0s * 1e6, 100 * ef_rhmax, label=r"$f_v = 1$ , max. RH", color="red")

plt.loglog(r0s * 1e6, 100 * ef, label=r"$f_v$ from PK97", color="brown")

plt.loglog(
    r0s * 1e6,
    100 * ef_ideal * fvs,
    label=r"analytic theory $\chi_{f_v = 1} f_v(r_0)$",
    linestyle="dotted",
    linewidth=1.5,
    color="green",
)

# plt.loglog(r0s*1e6, 100*ef_realreal, label = r"$f_v = 1$, realistic $C_D$", color = 'k', linestyle = "dashed")

# plt.plot(r0s*1e6, 100*1e-9*r0s**(-5/2))
# plt.plot(r0s*1e6, 100*8e-6*r0s**(-3/2))

plt.ylim(0.1, 120)
plt.xlim(6e1, 3e3)
plt.legend()
plt.xlabel(r"$r_0 \, (\mathrm{\mu m})$")
plt.ylabel(r"$\chi$ (%)")
plt.savefig("evapfraction_vs_r_onedropletmodel.pdf")

/var/folders/t3/z98pfw6x1rx6pfr0z0s_shd40000gn/T/ipykernel_34930/1077798909.py:3: RuntimeWarning: invalid value encountered in power
  efftheo = 1-(1-dr52/r0s**(5/2))**(6/5)
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp
'created' timestamp seems very low; regarding as unix timestamp
'modified' timestamp seems very low; regarding as unix timestamp


In [ ]:
plt.semilogx(r0s * 1e6, ef / ef_ideal, label=r"EF ratio with/without")
rosplot = np.logspace(-4, -2.5, 1000)
fvs = np.array([fv(r0, vtlim(r0)) for r0 in rosplot])
plt.plot(rosplot * 1e6, fvs, label=r"$f_v$")
plt.ylim(0, 20)
plt.legend()
plt.xlabel(r"$r_0 \, (\mathrm{\mu m})$")
plt.ylabel(r"EF ratio")
plt.savefig("EFratio_vs_r_onedropletmodel.png", dpi=600)

# Fv approxmiation

In [ ]:
rosplot = np.logspace(-7, -2.5, 1000)
fvs = np.array([fv(r0, vt(r0)) for r0 in rosplot])
plt.plot(rosplot * 1e6, fvs)
# fvs = np.array([fv(r0, vtlim(r0)) for r0 in rosplot])
plt.plot(rosplot * 1e6, fvs)

# plt.plot(rosplot*1e6, rosplot**(3/4)*1.5e3)
# plt.plot(rosplot*1e6, 1/(1/(rosplot**3*1.1e12)+1/(rosplot**(3/4)*1.3e3)), color = 'k')
plt.plot(rosplot * 1e6, (rosplot / 65e-6) ** (3 / 4), color="k")

plt.xlabel(r"$r \, (\mathrm{\mu m})$")
plt.ylabel(r"$f_v$")
# plt.loglog()
plt.xlim(0, 3200)
plt.ylim(0, 20)
plt.axhline(y=1, color="k", linestyle="dashed")
# plt.semilogx()
# plt.xlim(0, 3500)
# plt.ylim(0, 17)
# plt.loglog()
# plt.savefig("fv.png", dpi = 600)

In [ ]:
fv(2.2e-3, vtlim(2.2e-3))

15.938430606575267

# Terminal velocity?

In [ ]:
rfs = np.zeros_like(r0s)
list_simus = []
dt = 1e-3
time = 1e3
for k, r0 in enumerate(r0s):
    s = Simu_ideal(r0, rhosat_water(T), RH * rhosat_water(T))
    s.solve(H, time, dt)
    list_simus.append(s)
    rfs[k] = s.solution.y[-1, 0]
ef_ideal = (r0s**3 - rfs**3) / r0s**3

plt.plot(d_ef_cleo[:, 0], d_ef_cleo[:, 1], linestyle="", marker="+", markersize=5)

plt.loglog(r0s * 1e6, ef_ideal * 100)

In [ ]:
for k, r0 in enumerate(r0s):
    s = list_simus[k]
    plt.plot((s.solution.y[:, 1] / (bU * np.sqrt(s.solution.y[:, 0])) - 1) * 100, H - s.solution.x)
dx = 1e-3
plt.xlim(0, dx * 100)

(0.0, 0.1)

# OLD

In [ ]:
# %% Solution for various initial radii and saturations
list_a0 = np.logspace(-4, np.log10(5e-3), 20)
list_S = np.linspace(0.4, 0.9, 5)

list_simu = []
rf = []  # mass when the drop reaches the ground

dt = 1e-6
time = 1
H = 1000  # m


for i, a0 in enumerate(list_a0):
    list_simu.append([])
    rf.append([])
    for j, S in enumerate(list_S):
        s = Simu(a0, rhosat_water(T), S * rhosat_water(T))
        print(a0, S)
        s.solve(H, time, dt)
        list_simu[i].append(s)
        if not s.solution.evaporated and s.solution.touched_ground:
            rf[i].append(s.solution.y[-1, 0])
        elif not s.solution.evaporated and not s.solution.touched_ground:
            print(s.a0, s.S, "not touched ground ! iterate more")
        else:
            rf[i].append(0)

In [ ]:
# %% Plot the final mass as a function of a0 for different values of S
for k, l in enumerate(np.transpose(rf)):
    plt.semilogx(list_a0, l, label=list_S[k])
plt.legend(loc="best")

In [ ]:
plt.plot(list_simu[0][0].solution.t, list_simu[0][0].solution.y)

In [ ]:
# %% Plot the results in the a0, S plane
c = plt.contour(list_a0, 100 * (1 - list_S), np.transpose(rf) * 1e6, cmap="hsv")
plt.xscale("log")
plt.clabel(c)
plt.xscale("log")
plt.xlabel(r"$a_0\,(\mathrm{m})$")
plt.ylabel(r"$\mathrm{RH}\,(\%)$")

Text(0, 0.5, '$\\mathrm{RH}\\,(\\%)$')

In [ ]:
# %% Zeroth order solution for comparison
mf_0 = []

for i, a0 in enumerate(list_a0):
    mf_0.append([])
    for j, S in enumerate(list_S):
        s = list_simu[i][j]
        u = 2 * s.fv(1, 1) * Dv * Mmol * ns * S * H / (rhow * s.a0**2 * s.vt0)
        if u >= 1:
            mf_0[i].append(0)
        else:
            mf_0[i].append((1 - u) ** (3 / 2))

In [ ]:
# %% Compare zeroth order and numerical solution
levels = np.linspace(0.1, 0.95, 5)
cmap = "Dark2"
c = plt.contour(list_a0, 100 * (1 - list_S), np.transpose(mf), cmap=cmap, levels=levels)
plt.xscale("log")
plt.clabel(c)
c = plt.contour(
    list_a0, 100 * (1 - list_S), np.transpose(mf_0), linestyles="dashed", cmap=cmap, levels=levels
)
plt.xscale("log")
plt.xlabel(r"$a_0\,(\mathrm{m})$")
plt.ylabel(r"$\mathrm{RH}\,(\%)$")
# plt.clabel(c)

In [ ]:
# %% Superterminal velocity comparison
skip = 10  # skip points to display faster
for s in np.array(list_simu).flatten()[::3]:
    if not s.solution.evaporated and s.solution.touched_ground:
        ts = 2 * rhow * s.a0**2 / (9 * eta)
        a = (3 * s.solution.y[::skip, 0] * s.m0 / (4 * pi * rhow)) ** (1 / 3)
        plt.semilogy(
            s.solution.t[::skip],
            s.solution.y[::skip, 1] * s.vt0 / s.vt(a) - 1,
            label="S {:.2f} a0 {:.2e}".format(s.S, s.a0),
        )

plt.legend(loc="best")